In [ ]:
%scala
val storage_container = "data"
val storage_account_name = "<NAME>"
val storage_account_access_key = "<KEY>"
val storage_base_url = s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net"

val file_type = "csv"
val train_data_url = s"${storage_base_url}/data/train/train_data/*.${file_type}"
val  test_data_url = s"${storage_base_url}/data/test/test_data/*.${file_type}"

storage_container: String = data
storage_account_name: String = cs1100320004e0010e6
storage_account_access_key: String = an5UdS00UuMcb6vT54s2kWCF31rW++P9BKK7K0czqJDmONenltWmfatssB+4tAQOAHbYOUALp2RB+AStHj88Gg==
storage_base_url: String = wasbs://data@cs1100320004e0010e6.blob.core.windows.net
file_type: String = csv
train_data_url: String = wasbs://data@cs1100320004e0010e6.blob.core.windows.net/data/train/train_data/*.csv
test_data_url: String = wasbs://data@cs1100320004e0010e6.blob.core.windows.net/data/test/test_data/*.csv

In [ ]:
%scala
import org.apache.spark.sql.functions.{col, when, split, size, lower, concat_ws, regexp_replace, input_file_name}
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, StringType, DoubleType, ArrayType}
import org.apache.spark.sql.SparkSession

val spark = ( SparkSession.builder()
                          .appName("GloVe")
                          .config(s"fs.azure.account.key.${storage_account_name}.blob.core.windows.net", storage_account_access_key)
                          .getOrCreate() )
val sc = spark.sparkContext

import org.apache.spark.sql.functions.{col, when, split, size, lower, concat_ws, regexp_replace, input_file_name}
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, StringType, DoubleType, ArrayType}
import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@584ece16
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@70de5d63

In [ ]:
%scala
val stopWordsEN = Array(
 "i","me","my","myself",
 "we","our","ours","ourselves",
 "you","you're","you've","you'll","you'd","your","yours","yourself","yourselves",
 "he","him","his","himself",
 "she","she's","her","hers","herself",
 "it","it's","its","itself",
 "they","them","their","theirs","themselves",
 "what","which","who","whom",
 "this","that","that'll","these","those",
 "am","is","are","was","were","be","been","being",
 "have","has","had","having",
 "do","does","did","doing",
 "a","an","the",
 "of","at","by","for","with",
 "about","against","between","into","through","during",
 "before","after",
 "above","below",
 "to","from","up","down","in","out","on","off","over","under",
 "again","further","then","once","here","there",
 "when","where","why","how","all","any","both","each",
 "s","t",
 "can","will","just",
 "should","should've",
 "now","d","ll","m","o","re","ve","y","ma"
)

stopWordsEN: Array[String] = Array(i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, s, t, can, will, just, should, should've, now, d, ll, m, o, re, ve, y, ma)

In [ ]:
%scala
import com.johnsnowlabs.nlp.annotator.{DocumentNormalizer, Tokenizer, LemmatizerModel, WordEmbeddingsModel, StopWordsCleaner}
import com.johnsnowlabs.nlp.embeddings.SentenceEmbeddings
import com.johnsnowlabs.nlp.base.DocumentAssembler
import com.johnsnowlabs.nlp.EmbeddingsFinisher
import org.apache.spark.ml.Pipeline
import spark.implicits._

val documentAssembler = new DocumentAssembler()
  .setInputCol("comment")
  .setOutputCol("document")

val documentNormalizer = new DocumentNormalizer()
  .setInputCols("document")
  .setOutputCol("normalizedDocument")
  .setAction("clean")
  .setPatterns(Array("<[^>]>", """[^\w\d\s]"""))
  .setReplacement(" ")
  .setPolicy("pretty_all")
  .setLowercase(true)

val tokenizer = new Tokenizer()
  .setInputCols(Array("normalizedDocument"))
  .setOutputCol("token")

val stopWords = new StopWordsCleaner()
  .setInputCols("token")
  .setOutputCol("cleanTokens")
  .setStopWords(stopWordsEN)

val lemmatizer = LemmatizerModel.pretrained()
  .setInputCols(Array("cleanTokens"))
  .setOutputCol("lemma")

val embeddings = WordEmbeddingsModel.pretrained()
  .setInputCols("normalizedDocument", "cleanTokens")
  .setOutputCol("embeddings")

val embeddingsSentence = new SentenceEmbeddings()
  .setInputCols(Array("normalizedDocument", "embeddings"))
  .setOutputCol("sentence_embeddings")
  .setPoolingStrategy("AVERAGE")

val embeddingsFinisher = new EmbeddingsFinisher()
  .setInputCols("sentence_embeddings")
  .setOutputCols("finished_embeddings")
  .setOutputAsVector(false)

val pipeline = new Pipeline()
  .setStages(Array(
    documentAssembler,
    documentNormalizer,
    tokenizer,
    stopWords,
    lemmatizer,
    embeddings,
    embeddingsSentence,
    embeddingsFinisher
  ))

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
Download done! Loading the resource.
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
Download done! Loading the resource.
import com.johnsnowlabs.nlp.annotator.{DocumentNormalizer, Tokenizer, LemmatizerModel, WordEmbeddingsModel, StopWordsCleaner}
import com.johnsnowlabs.nlp.embeddings.SentenceEmbeddings
import com.johnsnowlabs.nlp.base.DocumentAssembler
import com.johnsnowlabs.nlp.EmbeddingsFinisher
import org.apache.spark.ml.Pipeline
import spark.implicits._
documentAssembler: com.johnsnowlabs.nlp.DocumentAssembler = document_575000d0fcb6
documentNormalizer: com.johnsnowlabs.nlp.annotators.DocumentNormalizer = DOCUMENT_NORMALIZER_6a2150b95c51
tokenizer: com.johnsnowlabs.nlp.annotators.Tokenizer = REGEX_TOKENIZER_aa479c89c16a
stopWords: com.johnsnowlabs.nlp.annotators.StopWordsCleaner = STOPWORDS_CLEANER_0ea3f6d742df
lemmatizer: com.johnsnowlabs.nlp.annotators.LemmatizerModel = LEMMATIZER_c62ad8f355f9
embeddings: com.johnsnowlabs.nlp.embeddings.WordEmbeddingsModel = WORD_EMBEDDINGS_MODEL_48cffc8b9a76
embeddingsSentence: com.johnsnowlabs.nlp.embeddings.SentenceEmbeddings = SENTENCE_EMBEDDINGS_d4eb4fc68606
embeddingsFinisher: com.johnsnowlabs.nlp.EmbeddingsFinisher = embeddings_finisher_1850d5ce3e78
pipeline: org.apache.spark.ml.Pipeline = pipeline_03463de6729e

In [ ]:
%scala
val schema = StructType(
  StructField("id"     , IntegerType, true) ::
  StructField("rating" , IntegerType, true) ::
  StructField("type"   , IntegerType, true) ::
  StructField("comment", StringType , true) :: Nil
)

val trainDF = spark.read.format("csv")
                        .schema(schema)
                        .option("header","true")
                        .load(train_data_url)
                        .filter( col("type") =!= -1 )
                        .cache()
val testDF  = spark.read.format("csv")
                        .schema(schema)
                        .option("header","true")
                        .load(test_data_url)
                        .filter( col("type") =!= -1 )
                        .cache()

val transformedTrainDF = pipeline.fit(trainDF)
                                 .transform(trainDF)
                                  .select(
                                    col("id"),
                                    col("rating"),
                                    col("type").as("sentiment"),
                                    col("finished_embeddings")(0).as("wordEmbedding")
                                  )
                                 .cache()
val transformedTestDF  = pipeline.fit(testDF)
                                 .transform(testDF)
                                  .select(
                                    col("id"),
                                    col("rating"),
                                    col("type").as("sentiment"),
                                    col("finished_embeddings")(0).as("wordEmbedding")
                                  )
                                 .cache()

schema: org.apache.spark.sql.types.StructType = StructType(StructField(id,IntegerType,true),StructField(rating,IntegerType,true),StructField(type,IntegerType,true),StructField(comment,StringType,true))
trainDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, rating: int ... 2 more fields]
testDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, rating: int ... 2 more fields]
transformedTrainDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, rating: int ... 2 more fields]
transformedTestDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, rating: int ... 2 more fields]

In [ ]:
%scala
transformedTrainDF.show(5, 80)

+-----+------+---------+--------------------------------------------------------------------------------+
 id|rating|sentiment| wordEmbedding|
+-----+------+---------+--------------------------------------------------------------------------------+
 1175| 9| 1|[0.013433628, 0.008061813, 0.19747755, -0.31950152, -0.035062358, 0.13085975,...|
 4383| 9| 1|[-0.017688854, 0.17036815, 0.28469002, -0.23391548, -0.03514681, 0.18845885, ...|
10044| 9| 1|[-0.03156714, 0.18471205, 0.22300884, -0.15945582, 0.028217627, 0.057009477, ...|
 4076| 10| 1|[-0.07171856, 0.20415959, 0.289257, -0.18681042, 0.050364405, 0.13820319, -0....|
 2662| 10| 1|[-0.025108574, 0.15275864, 0.22979191, -0.19176292, 0.03289066, 0.1865619, -0...|
+-----+------+---------+--------------------------------------------------------------------------------+
only showing top 5 rows

In [ ]:
%scala
transformedTestDF.show(5, 80)

+----+------+---------+--------------------------------------------------------------------------------+
 id|rating|sentiment| wordEmbedding|
+----+------+---------+--------------------------------------------------------------------------------+
8647| 10| 1|[-0.016145764, 0.146732, 0.28266045, -0.15704648, 0.008063161, 0.20166203, -0...|
3801| 7| 1|[-0.03836236, 0.24948506, 0.29505295, -0.124756105, -0.012420236, 0.22264947,...|
4051| 10| 1|[-0.045071196, 0.21207888, 0.23317057, -0.28309605, -0.053015966, 0.16963571,...|
 754| 9| 1|[-0.035816267, 0.1980528, 0.23870033, -0.17985478, -0.020196006, 0.20412652, ...|
3958| 9| 1|[-8.468482E-4, 0.16253671, 0.2643108, -0.22896665, 0.028373038, 0.23134348, -...|
+----+------+---------+--------------------------------------------------------------------------------+
only showing top 5 rows

In [ ]:
%scala
transformedTrainDF
       .repartition(1)
       .write
       .mode("overwrite")
       .format("parquet")
       .save(s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net/data/train/WordEmbedding_train")

transformedTestDF
       .repartition(1)
       .write
       .mode("overwrite")
       .format("parquet")
       .save(s"wasbs://${storage_container}@${storage_account_name}.blob.core.windows.net/data/test/WordEmbedding_test")